## Ejercicio de Tarea 05
Para los datos del ejercicio 1 de la liga de Futbol

**a**. Usar el algoritmo de selección hacia adelante para seleccionar un modelo de regresión.

**b**. Usar el algoritmo de selección hacia atrás para seleccionar un modelo de regresión.

**c**. Usar el algoritmo de regresión por pasos para seleccionar un modelo de regresión.

**d**. Comenta los modelos finales en cada uno de los casos anteriores. ¿Cuál tiene más sentido? ¿Cuál modelo usarían? 

## 0. Importar 

### 0.1. Importar Librerías

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import statsmodels.formula.api as smf

from scipy import stats

import SourcePython as src

### 0.2. Cargar Datos

In [3]:
DatasetFutbol = pd.read_csv(
    './Liga_nacional_de_futbol.csv',
)

TargetLabel , *FeatureLabels = DatasetFutbol.columns

## **a**. Modelo por Selección hacia Adelante

In [ ]:
def GenerateModels(
        Dataset: pd.DataFrame,
        TargetLabel: str,
    ):
    """
    Función para facilitar la creación y 
    generación de los modelos en base a 
    un conjunto de datos.
    """

    def CreateModelInstance(
            FeaturesModel: list[str]
        ):
        """
        Función que crea los modelos en 
        base a los atributos que se 
        están considerando
        """

        LinearModel = smf.ols(
            f"{TargetLabel} ~ " + ' + '.join(FeaturesModel),
            data = Dataset,
        ).fit()

        return LinearModel
    
    return CreateModelInstance

def EvaluateModel(
        LinearModel
    ) -> float:
    """
    Función para obtener la métrica de evaluación de un modelo. 
    Esta métrica se está minimizando por los diferentes métodos.
    """ 

    return LinearModel.mse_resid

CreateModelInstance = GenerateModels(DatasetFutbol,TargetLabel)

In [ ]:
from copy import deepcopy

def ForwardSelection(
        Dataset: pd.DataFrame,
        FeatureLabels: list[str],
        TargetLabel: str,
    ) -> list[str]: 
    """
    Algoritmo para la selección hacia delante
    """

    CreateModelInstance = GenerateModels(Dataset,TargetLabel)
    BestLinearModel = smf.ols(
        f"{TargetLabel} ~ 1",
        data = Dataset
    ).fit()
    BestScore = EvaluateModel(BestLinearModel)
    BestFeatures = []

    AvailableFeatures = deepcopy(FeatureLabels)
    while True:
        best_score_alt = np.inf
        best_feature = ''
        for feature in AvailableFeatures:
            LinearModel_Alt = CreateModelInstance(BestFeatures+[feature])
            score = EvaluateModel(LinearModel_Alt)

            if score < best_score_alt:
                best_feature = feature
                best_score_alt = score

        if best_score_alt < BestScore:
            print(f"ADD :: {best_feature}")
            BestFeatures.append(best_feature)
            BestScore = best_score_alt
            AvailableFeatures.remove(best_feature)
        else:
            return BestFeatures

In [8]:
best_features_forward = ForwardSelection(DatasetFutbol,FeatureLabels,TargetLabel)
EvaluateModel(CreateModelInstance(best_features_forward)) , best_features_forward

(np.float64(2.8262759381660296), ['x8', 'x2', 'x7', 'x9'])

## **b**. Modelo por Selección hacia Atrás

In [9]:
from copy import deepcopy

def BackwardSelection(
        Dataset: pd.DataFrame,
        FeatureLabels: list[str],
        TargetLabel: str,
    ) -> list[str]: 
    """
    Algoritmo para la selección hacia atrás
    """

    CreateModelInstance = GenerateModels(Dataset,TargetLabel)
    BestLinearModel = CreateModelInstance(FeatureLabels)
    BestScore = EvaluateModel(BestLinearModel)
    BestFeatures = deepcopy(FeatureLabels)

    while True:
        best_score_alt = np.inf
        worst_feature = ''
        for feature in BestFeatures:
            LinearModel_Alt = CreateModelInstance([__feature for __feature in BestFeatures if __feature != feature])
            score = EvaluateModel(LinearModel_Alt)
            
            if score < best_score_alt:
                worst_feature = feature
                best_score_alt = score

        if best_score_alt < BestScore:
            print(f"REMOVE :: {worst_feature}")
            BestScore = best_score_alt
            BestFeatures.remove(worst_feature)
        else:
            return BestFeatures

In [10]:
best_features_backward = BackwardSelection(DatasetFutbol,FeatureLabels,TargetLabel)
EvaluateModel(CreateModelInstance(best_features_backward)) , best_features_backward

REMOVE :: x5
REMOVE :: x1
REMOVE :: x6
REMOVE :: x3
REMOVE :: x4


(np.float64(2.826275938166031), ['x2', 'x7', 'x8', 'x9'])

## **c**. Modelo por Selección por Pasos

In [11]:
from copy import deepcopy

def StepwiseSelection(
        Dataset: pd.DataFrame,
        FeatureLabels: list[str],
        TargetLabel: str,
    ) -> list[str]: 
    """
    Algoritmo para la selección por pasos
    """
    
    CreateModelInstance = GenerateModels(Dataset,TargetLabel)
    BestLinearModel = smf.ols(
        f"{TargetLabel} ~ 1",
        data = Dataset
    ).fit()
    BestScore = EvaluateModel(BestLinearModel)
    BestFeatures = []

    while True:
        best_score = BestScore
        trial_features = deepcopy(BestFeatures)

        best_add_feature = ''
        for feature in [_feature for _feature in FeatureLabels if _feature not in trial_features]:
            LinearModel_Alt = CreateModelInstance(trial_features+[feature])
            score = EvaluateModel(LinearModel_Alt)
            
            if score < best_score:
                best_add_feature = feature
                best_score = score

        if best_add_feature:
            print(f"ADD :: {best_add_feature}")
            trial_features.append(best_add_feature)

        worst_remove_feature = ''
        for feature in trial_features:
            subset_features = [_feature for _feature in trial_features if _feature != feature]
            if subset_features:
                LinearModel_Alt = CreateModelInstance(subset_features)
                score = EvaluateModel(LinearModel_Alt)

                if score < best_score:
                    worst_remove_feature = feature
                    best_score = score

        if worst_remove_feature: 
            print(f"REMOVE :: {worst_remove_feature}")
            trial_features.remove(worst_remove_feature)

        LinearModel_Alt = CreateModelInstance(trial_features)
        best_score = EvaluateModel(LinearModel_Alt)
        if best_score < BestScore:
            BestFeatures = deepcopy(trial_features)
            BestScore = best_score
        else:
            break
    
    return BestFeatures

In [12]:
best_features_stepwise = StepwiseSelection(DatasetFutbol,FeatureLabels,TargetLabel)
EvaluateModel(CreateModelInstance(best_features_stepwise)) , best_features_stepwise 

ADD :: x8
ADD :: x2
ADD :: x7
ADD :: x9


(np.float64(2.8262759381660296), ['x8', 'x2', 'x7', 'x9'])

## **d**. Comentarios Finales sobre los Modelos Generados

La parte relevante de estos métodos son tanto las decisiones que toman para llegar a las variables finales y las propias métricas evaluación o criterio para escoger el mejor modelo.

Para el modelo de selección hacia adelante, las variables son añadidas en el siguiente orden: $x8$ (Yardas por tierra del contrario), $x2$ (Yardas por aire), $x7$ (Porcentaje de carreras) y $x9$ (Yardas por aire del contrario). Estas variables muestran qué tan agresivo es el equipo contrario y la ofensa del equipo, estos dos aspectos determinan si se gana o no un partido.

Para el modelo de selección hacia atrás, las variables son eliminadas en el siguiente orden: $x5$ (Diferencia de pérdidas de balón), $x1$ (Yardas por tierra), $x6$ (Yardas de castigo), $x3$ (Promedio de pateo) y $x4$ (Porcentaje de goles de campo). Estas variables engloban aspectos como la cantidad de intentos de goles, diferencia en los puntajes y otras penalización que pueden verse como aspectos aleatorios si se consideran muchos juegos.

Para el modelo de selección por pasos sigue el mismo patrón que el modelo de selección hacia adelante, es decir, no realiza eliminaciones de atributos y los añade en el mismo orden.

Para este caso en particular, se llegó al mismo modelo. Pero por cómo se construye, el modelo en selección por pasos se tiene un modelo más robusto que considera el cómo el añadir un nuevo atributo puede alterar al propio modelo; esto hace que tenga más sentido sobre las selecciones que realice, debido a la constante reconsideración que hace. Por ello, me quedaría con los modelos generados a partir de éste método.